# comparison of accessibility of nucleotides in the sequences

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
import sys
sys.path.insert(0,'..')
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')
import matplotlib.pyplot as plt

# Load data

In [ ]:
import rna_tools.rna_io as rio

def getdata():
    return rio.get_all_data("../data/RNA16.react","../data/RNA16.dbn")  # {key: rea, seq, stru}

data = getdata()

for k,v in data.items():print k,"\t",  len(v[1])

# train predict

In [ ]:
import simushape as sim
from rna_tools.rnaplfold import rnaplfold

data = getdata()
model  = sim.make_model(data,['23sRNA','R009'])
data.pop("23sRNA")
data.pop("R009")

keys = data.keys()

def get_str(seq):
    for _, data_seq, db in data.values():
        if seq == data_seq:
            return [db]
    print "SHIT"
    
predictions = { k: (data[k][1],sim.predict(model,data[k][1])) for k in keys  }
predictions_str = { k: (data[k][1],sim.predict(model,data[k][1], seq_to_db_function=get_str)) for k in keys  }
acc_shaker = {k: rnaplfold(*predictions[k]) for  k in keys} # a predcition is seq,rea
acc_shaker_str = {k: rnaplfold(*predictions_str[k]) for  k in keys} # a predcition is seq,rea


In [ ]:
if False:
    data = getdata()
    keys = data.keys()
    shaker_shape = sim.crosspredict_nfold(data,keys)
    

# real and sukosd based accessibility

In [ ]:
from rna_tools.sukosd import sukosd
acc_suko =   {k: rnaplfold(data[k][1], sukosd(data[k][2]) ) for  k in keys}
acc_shape =  {k: rnaplfold(data[k][1],data[k][0]) for  k in keys}  
acc_nodata = {k: rnaplfold(data[k][1]) for  k in keys}


acc_theoretical_best = {k: rnaplfold(data[k][1],[1.0 if e =='.' else 0.0 for e in data[k][2]]) for k in keys} ### DOES THIS WORK? 


In [ ]:
from scipy.stats import spearmanr as spear
corr = lambda ac1, ac2:[spear(ac1[k], ac2[k])[0] for k in keys]
corr_suko =  corr(acc_suko, acc_shape)
corr_shaker =  corr(acc_shaker, acc_shape)
corr_shaker_str =  corr(acc_shaker_str, acc_shape)
corr_nodata =  corr(acc_nodata, acc_shape)
corr_theo = corr(acc_theoretical_best, acc_shape)

# Visualize 

In [ ]:
import pandas
import numpy as np
index= ['Sukosd',"ShaKer+Structure", "known_strucure_max","ShaKer","Thermodynamic"]

data2 = [corr_suko, corr_shaker_str,corr_theo,corr_shaker,corr_nodata]


import seaborn as sns
sns.boxplot(data=pandas.DataFrame(data2[:3], index=index[:3]).T)
TITLE = "Accessibility"
plt.title("SOME TITLE")
plt.grid()

In [ ]:
import seaborn as sns
sns.boxplot(data=pandas.DataFrame(data2[3:], index=index[3:]).T)
plt.title("SOME TITLE")
plt.grid()

In [ ]:
plt.xticks(rotation=70)
g=sns.boxplot(data=pandas.DataFrame(data2, index=index).T)
plt.title(TITLE)
g.set(ylim=(0, 1))
plt.grid()

In [ ]:
plt.xticks(rotation=70)
g=sns.stripplot(data=pandas.DataFrame(data2, index=index).T)
g=sns.boxplot(data=pandas.DataFrame(data2, index=index).T)
plt.title(TITLE)
g.set(ylim=(0, 1))
plt.grid()

In [ ]:
map(lambda x:x.append(np.mean(x)),data2)    
data2 = pandas.DataFrame(data2, columns=keys+['mean'], index=index).T
from tabulate import tabulate
print tabulate(data2, headers="keys", tablefmt="orgtbl")

In [ ]:
print data2.to_latex()

In [ ]:
"""
from scipy.stats import spearmanr as spear
corr = lambda ac1, ac2:[(a-b)**2 for k in keys for a,b in zip(ac1[k], ac2[k])]
corr_suko =  corr(acc_suko, acc_shape)
corr_shaker =  corr(acc_shaker, acc_shape)
corr_shaker_str =  corr(acc_shaker_str, acc_shape)
corr_nodata =  corr(acc_nodata, acc_shape)
print len(corr_suko)

import pandas
index= ['suko',"shaker","shaker_str","nodata"]
data2 = [corr_suko,corr_shaker, corr_shaker_str,corr_nodata]
data2 = pandas.DataFrame(data2, index=index).T
print data2.plot(kind='box')
"""

---